# Text Search Engine
In this example we will be going over the code used to build a Text Search Engine. This example uses a modified BERT model to convert text to vectors stored in Milvus, which can then be combined with Milvus to search for similar text to the user input text.

## Data

This example uses the English News dataset. In this example, we use a small subset of the dataset containing 180 mutually corresponding title-texts, which can be found in the **Data** directory.

## Requirements


|  Packages   |  Servers    |
|-                  | -                 |   
| pymilvus==2.0.0rc5      | milvus 2.0.0-rc5    |
| sentence_transformers      | postgres          |
| psycopg2          |
| pandas           |
| numpy   |

We have included a `requirements.txt` file in order to easily satisfy the required packages. 


## Up and Running

### Installing Packages
Install the required python packages with `requirements.txt`.

In [1]:
pip install -r requirements.txt

### Starting Milvus Server

This demo uses Milvus 2.0, please refer to the [Install Milvus](https://milvus.io/cn/docs/install_standalone-docker.md) guide to learn how to use this docker container. For this example we wont be mapping any local volumes. 

In [4]:
!wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml
!sudo docker-compose up -d

### Starting Postgres Server
For now, Milvus doesn't support storing multiple attributes for the data. Because of this we have to use another service to store these attributes and search through them, in this case PostgreSQL. 

In [15]:
! docker run --name postgres0 -d  -p 5438:5432 -e POSTGRES_HOST_AUTH_METHOD=trust postgres

In [6]:
! docker logs postgres0 --tail 6

## Code Overview

### Connecting to Servers
We first start off by connecting to the servers. In this case the docker containers are running on localhost and the ports are the default ports. 

In [16]:
#Connectings to Milvus, BERT and Postgresql
from pymilvus import connections
import psycopg2
connections.connect(host='localhost', port='19530')
conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
cursor = conn.cursor()

### Creating Collection and Setting Index
#### 1. Creating the Collection    
The next step is to create a collection, which requires declaring the name of the collection and the dimension of the vector.

In [6]:
TABLE_NAME = "text_collection"
field_name = "example_field"
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType
pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
field = FieldSchema(name=field_name, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="example collection")
collection = Collection(name=TABLE_NAME, schema=schema)

#### 2. Setting an Index
After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_SQ8 which requires the 'nlist' parameter.

In [43]:
index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }
collection.create_index(field_name=field_name, index_params=index_param)

Status(code=0, message='')

### Creating Table in Postgres  
PostgresSQL will be used to store Milvus ID and its corresponding title and text.

In [47]:
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME
cursor.execute(drop_table)
conn.commit()

try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (ids bigint, title text, text text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

create postgres table successfully!


### Processing and Storing the News Data
#### 1. Generating Embeddings
In this example we are using the sentence_transformer library  to encode the sentence into vectors. This library uses a modified BERT model to generate the embeddings, and in this example we are using a model pretrained using Microsoft's `mpnet`. More info can be found [here](https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models).

In [48]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

model = SentenceTransformer('paraphrase-mpnet-base-v2')
# Get questions and answers.
data = pd.read_csv('data/example.csv')
title_data = data['title'].tolist()
text_data = data['text'].tolist()

sentence_embeddings = model.encode(title_data)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

You try to use a model that was created with version 1.2.0, however, your version is 1.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





<class 'numpy.ndarray'>


#### 2. Inserting Vectors into Milvus
Since this example dataset contains only 100 vectors, we are inserting all of them as one batch insert.

In [49]:
em =list(sentence_embeddings)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

#### 3. Inserting IDs and Title-text into PostgreSQL
In order to transfer the data into Postgres, we are creating a new file that combines all the data into a readable format. Once created, we pass this file into the Postgress server through STDIN due to the Postgres container not having access to the file locally. 

In [50]:
import os 

def record_temp_csv(fname, ids, title, text):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + title[i] + "|" + text[i] + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    try:
        sql = "COPY " + table_name + " FROM STDIN DELIMITER '|' CSV HEADER"
        cursor.copy_expert(sql, open(fname, "r"))
        conn.commit()
        print("Inserted into Postgress Sucessfully!")
    except Exception as e:
        print("Copy Data into Postgress failed: ", e)
        
DATA_WITH_IDS = 'data/test.csv'   

record_temp_csv(DATA_WITH_IDS, ids, title_data, text_data)
copy_data_to_pg(TABLE_NAME, DATA_WITH_IDS, conn, cursor)

Inserted into Postgress Sucessfully!


### Search
#### 1. Processing Query
When searching for a question, we first put the question through the same model to generate an embedding. Then with that embedding vector we  can search for similar embeddings in Milvus.  

In [51]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

query_vec = []

title = "Loosing the War on Terrorism"

query_embeddings = []
embed = model.encode(title)
embed = embed.reshape(1,-1)
embed = normalize(embed)
query_embeddings = embed.tolist()

collection.load()
results = collection.search(query_embeddings, field_name, param=search_params, limit=9, expr=None)

#### 2. Getting the Similar Titles
There may not have titles that are similar to the given one. So we can set a threshold value, here we use 0.5, and when the most similar distance retrieved is less than this value, a hint that the system doesn't include the relevant question is returned. We then use the result ID's to pull out the similar titles from the Postgres server and print them with their corresponding similarity score.

In [53]:
similar_titles = []

if results[0][0].distance < 0.5:
    print("There are no similar questions in the database, here are the closest matches:")
else:
    print("There are similar questions in the database, here are the closest matches: ")
    
for result in results[0]:
    sql = "select title from " + TABLE_NAME + " where ids = " + str(result.id) + ";"
    cursor.execute(sql)
    rows=cursor.fetchall()
    if len(rows):
        similar_titles.append((rows[0][0], result.distance))
        print((rows[0][0], result.distance))
       

There are no similar questions in the database, here are the closest matches:
('Loosing the War on Terrorism', 4.2543743593130567e-13)
('Politics an Afterthought Amid Hurricane  ', 1.4113149642944336)
('Kerry-Kerrey Confusion Trips Up Campaign  ', 1.4152636528015137)
('News: Sluggish movement on power grid cyber security', 1.4480023384094238)
('Promoting a Shared Vision', 1.4696528911590576)
('U.S. Brokers Cease-fire in Western Afghanistan', 1.473806619644165)
('On front line of AIDS in Russia', 1.493909478187561)
('Fresh Fighting Shatters Short-Lived Ceasefire Deal', 1.5365568399429321)
('Flop in the ninth inning sinks Jays', 1.568418264389038)


#### 3. Get the text
After getting a list of similar titles, choose the one that you feel is closest to yours. Then you can use that title to find the corresponding text in Postgres.

In [54]:
sql = "select text from " + TABLE_NAME + " where title = '" + similar_titles[0][0] + "';"
cursor.execute(sql)
rows=cursor.fetchall()
print("Title:")
print(title)
print("Text:")
print(rows[0][0])

Title:
Loosing the War on Terrorism
Text:
 Sven Jaschan, self-confessed author of the Netsky and Sasser viruses, is responsible for 70 percent of virus infections in 2004, according to a six-month virus roundup published Wednesday by antivirus company Sophos.  The 18-year-old Jaschan was taken into custody in Germany in May by police who said he had admitted programming both the Netsky and Sasser worms, something experts at Microsoft confirmed. (A Microsoft antivirus reward program led to the teenager's arrest.) During the five months preceding Jaschan's capture, there were at least 25 variants of Netsky and one of the port-scanning network worm Sasser.  Graham Cluley, senior technology consultant at Sophos, said it was staggeri   
